In [1]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

Text for the predictor is in the <code>document</code> variable.

In [41]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")
#predictorOutput = predictor.predict(document="What is your recommended color? The recommended color is red. Please use it.")
predictorOutput = predictor.predict(document="Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers.")

In [42]:
def generateClusters(predictorOutput, currSentence):
    #function borrowed from: https://github.com/ananyagup/AllenNLP-Coreference-Resolution-in-Python-Readable-clusters
    clusters = predictorOutput['clusters']
    document = predictorOutput['document']
    n = 0
    doc = {}
    for obj in document:
        doc.update({n :  obj}) #what I'm doing here is creating a dictionary of each word with its respective index, making it easier later.
        n += 1
    clus_all = []
    cluster = []
    sublist = []
    print(doc)
    for i in range(0, len(clusters)):
        one_cl = clusters[i]
        for count in range(0, len(one_cl)):
            obj = one_cl[count]
            for num in range((obj[0]), (obj[1]+1)):
                for n in doc:
                    if num == n:
                        sublist.append(doc[n])
                        if n == obj[1]:
                            cluster.append(sublist)
                            sublist = []
        clus_all.append(cluster)
        cluster = []

    print(clus_all) #shows all coreferences
    return clus_all

In [43]:
# generateReferences takes in a context (1+ previous sentences)
# and a current sentence and replaces all broad references with  
# coreferences and the broad reference itself
# TODO: extend list of broad references
broad_refs = ["he", "she", "this", "that", "them", "it"]
def generateReferences(context, currSentence):
    perms = []
    modified = []
    clus_all = generateClusters(predictorOutput, currSentence)
    for cluster in clus_all:
        newW = ""
        print("cluster: ", cluster)
        for index, word in enumerate(cluster):
            print("word: ", word)
            if word[0] in broad_refs: #only supports a broad reference that is 1 word
                for newWord in cluster:
                    if len(newWord) == 1:
                        if newWord not in broad_refs:
                            modified.append(currSentence.replace(word[0], newWord[0]))
                    else:
                        newW =  ' '.join(newWord)
                        modified.append(currSentence.replace(word[0], newW))
                print(modified)
                #modified = [currSentence.replace(word, newWord) for newWord in cluster if newWord not in broad_refs]
                return list(set(modified))
                    
                    
# example: "Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers."
print("modified sentence: ", generateReferences("Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers.", "with whom he shared an enthusiasm for computers."))

# Bug found: output is modified sentence:  ['Please use your.', 'Please use color.', 'Please use The.', 'Please use recommended.']
# This is due to coreferences being > 1 word long... Need to modify generateClusters function to group together. 
#print("modified sentence: ", generateReferences("What is your recommended color? The recommended color is red. Please use it.", "Please use it."))
    

{0: 'Paul', 1: 'Allen', 2: 'was', 3: 'born', 4: 'on', 5: 'January', 6: '21', 7: ',', 8: '1953', 9: ',', 10: 'in', 11: 'Seattle', 12: ',', 13: 'Washington', 14: ',', 15: 'to', 16: 'Kenneth', 17: 'Sam', 18: 'Allen', 19: 'and', 20: 'Edna', 21: 'Faye', 22: 'Allen', 23: '.', 24: 'Allen', 25: 'attended', 26: 'Lakeside', 27: 'School', 28: ',', 29: 'a', 30: 'private', 31: 'school', 32: 'in', 33: 'Seattle', 34: ',', 35: 'where', 36: 'he', 37: 'befriended', 38: 'Bill', 39: 'Gates', 40: ',', 41: 'two', 42: 'years', 43: 'younger', 44: ',', 45: 'with', 46: 'whom', 47: 'he', 48: 'shared', 49: 'an', 50: 'enthusiasm', 51: 'for', 52: 'computers', 53: '.'}
placing delimeter
placing delimeter
placing delimeter
placing delimeter
placing delimeter
placing delimeter
[[['Paul', 'Allen'], ['Allen'], ['he'], ['he']], [['Seattle', ',', 'Washington'], ['Seattle']]]
cluster:  [['Paul', 'Allen'], ['Allen'], ['he'], ['he']]
word:  ['Paul', 'Allen']
word:  ['Allen']
word:  ['he']
['with whom Paul Allen shared an ent